In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
import pickle
import re
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
from nltk.corpus import wordnet
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
# import dask.dataframe as dd
# from dask.multiprocessing import get
import swifter
from tqdm import tqdm

tqdm.pandas()
lemmatizer = WordNetLemmatizer() 

In [2]:
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])


In [3]:

url_pattern = re.compile(r'https?://\S+|www\.\S+')
alpha_pattern = re.compile(r'[^A-Za-z ]+')
def clean_up(x):
    x = url_pattern.sub(r'', x)
    x = alpha_pattern.sub(r'', x)
    word_tokens = word_tokenize(x.lower())
    filtered_sentence =[]
    for w in word_tokens:
        if len(w)<3 or w in STOP_WORDS:
            continue
        filtered_sentence.append(w.lower())
    x = []
    for word in filtered_sentence:
        x.append(lemmatize_words(word))
#         x.append(ps.stem(word))
    x = " ".join(x)
#     x = lemmatize_words(x)
    return x

In [4]:
df = pd.read_csv("data/drug_review/drugsComTrain_raw.tsv",delimiter="\t")[:11000]
test = pd.read_csv("data/drug_review/drugsComTest_raw.tsv",delimiter="\t")[:4000]
break_point = df.shape[0]

In [5]:
df.review = df.review.swifter.allow_dask_on_strings().apply((lambda row: clean_up(row)))

In [6]:
test.review = test.review.swifter.allow_dask_on_strings().apply(lambda row: clean_up(row))

In [7]:
reviews = pd.concat([df.review,test.review])

In [8]:
from collections import Counter
cnt = Counter()
for text in tqdm(reviews.values):
    old_words = []
    for word in text.split():
        if word not in old_words:
            cnt[word] += 1
            old_words.append(word)


100%|█████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 37072.37it/s]


In [9]:
def remove_rare_freq_words(x):
    word_tokens = word_tokenize(x)
    x = []
    for w in word_tokens:
#         print("_*_"*100)
        if cnt[w] < high_pruner:
            x.append(w)
        elif cnt[w] > 20:
            x.append[w]
    x = " ".join(x)
    return x

In [10]:
high_pruner = reviews.shape[0]*0.65
reviews = reviews.progress_apply(lambda x : remove_rare_freq_words(x))
reviews[15]

100%|██████████████████████████████████████████████████████████████████████████| 15000/15000 [00:03<00:00, 4235.21it/s]


15    take saxenda july severe nausea month get dosa...
15    doctor prescribed provigil diagnose narcolepsy...
Name: review, dtype: object

In [11]:
clean_up("I happy had birthday too soon when where ok damn sorry after")

'happy birthday soon damn sorry'

#1. Dict
#2. Doc -> tokens token2id ##also remove duplicates
#3. Doc -> token_count 

In [12]:

texts = [[text for text in review.split()] for review in reviews]
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(19856 unique tokens: ['bystolic', 'combination', 'effect', 'fish', 'oil']...)


In [13]:
df.review = reviews[:break_point]
test.review = reviews[break_point:]

In [14]:
def prepare_df(df,col="review"):
    df = pd.DataFrame(reviews,columns=[col])
    df["bow"] = df[col].apply(lambda x : dictionary.doc2bow(x.split(),))
    df["tokens"] = df.bow.apply(lambda x: np.array([[i[0] for i in x]]))
    df["counts"] = df.bow.apply(lambda x: np.array([[i[1] for i in x]]))
    df.drop(['bow','review'],axis=1,inplace=True)
    return df

In [15]:
df = prepare_df(df)
df.head()

,tokens,counts
0,"[[0, 1, 2, 3, 4]]","[[1, 1, 1, 1, 1]]"
1,"[[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...","[[1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1,..."
2,"[[2, 12, 21, 22, 37, 41, 43, 44, 48, 50, 51, 5...","[[2, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,..."
3,"[[12, 62, 71, 74, 81, 82, 83, 84, 85, 86, 87, ...","[[3, 1, 1, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,..."
4,"[[22, 23, 41, 49, 96, 97, 98, 99, 100, 101, 10...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


In [16]:
train , valid = df.iloc[:int(df.shape[0]*0.9),:],df.iloc[int(df.shape[0]*0.9):,:]
print(train.shape)
print(valid.shape)

(13500, 2)
(1500, 2)


In [17]:
test = prepare_df(test)
print(test.shape)
test.head()

(15000, 2)


,tokens,counts
0,"[[0, 1, 2, 3, 4]]","[[1, 1, 1, 1, 1]]"
1,"[[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...","[[1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1,..."
2,"[[2, 12, 21, 22, 37, 41, 43, 44, 48, 50, 51, 5...","[[2, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,..."
3,"[[12, 62, 71, 74, 81, 82, 83, 84, 85, 86, 87, ...","[[3, 1, 1, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,..."
4,"[[22, 23, 41, 49, 96, 97, 98, 99, 100, 101, 10...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


In [18]:
test , test_1,test_2 = test.iloc[:int(test.shape[0]*0.5),:],test.iloc[int(test.shape[0]*0.5):int(test.shape[0]*0.75),:],test.iloc[int(test.shape[0]*0.75):,:]
print(test.shape)
print(test_1.shape)
print(test_2.shape)
# test_1_tokens = test['tokens_1']
# test_1_counts = test['counts_1']

(7500, 2)
(3750, 2)
(3750, 2)


In [45]:
df = pd.DataFrame()
df["review"] = reviews
print(df.shape)
df.dropna(inplace=True)
print(df.shape)
df.to_csv("data/drug_review/preprocessed_reviews.csv",index=None)

df = pd.read_csv("data/drug_review/preprocessed_reviews.csv")
df

(15000, 1)
(15000, 1)


,review
0,effect combination bystolic fish oil
1,son halfway fourth week intuniv concerned begi...
2,oral contraceptive pill cycle happy light peri...
3,time form birth control glad go patch month de...
4,suboxone completely turn life feel healthier e...
5,day start work rock hard erection experianced ...
6,pull cummed bit take plan hour later take preg...
7,abilify change life hope zoloft clonidine star...
8,problem keppera constant shake arm amp leg amp...
9,pill year doctor change chateal effective help...


In [21]:
train

,tokens,counts
0,"[[0, 1, 2, 3, 4]]","[[1, 1, 1, 1, 1]]"
1,"[[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...","[[1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1,..."
2,"[[2, 12, 21, 22, 37, 41, 43, 44, 48, 50, 51, 5...","[[2, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,..."
3,"[[12, 62, 71, 74, 81, 82, 83, 84, 85, 86, 87, ...","[[3, 1, 1, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,..."
4,"[[22, 23, 41, 49, 96, 97, 98, 99, 100, 101, 10...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
5,"[[2, 12, 41, 48, 93, 140, 141, 142, 143, 144, ...","[[2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 1, 1, 1,..."
6,"[[28, 43, 48, 163, 164, 165, 166, 167, 168, 16...","[[1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1]]"
7,"[[2, 41, 45, 49, 57, 77, 82, 89, 109, 116, 152...","[[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 4, 1, 1,..."
8,"[[35, 69, 109, 141, 158, 179, 201, 202, 203, 2...","[[1, 1, 1, 5, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1]]"
9,"[[7, 14, 17, 18, 41, 43, 44, 75, 81, 82, 93, 1...","[[1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2,..."


## Saving The Files

In [22]:
path = "data/drug_review/"

In [23]:
# train.to_csv(path+"train.csv",index=None)
# test.to_csv(path+"test.csv",index=None)
# valid.to_csv(path+"valid.csv",index=None)
dictionary.save(path+"vocab.pkl")

In [24]:
def store_bow_files(df,filename):
    tokens = df.tokens.tolist()
    counts = df.counts.tolist()
    dict_ = {"tokens":tokens,"counts":counts}
    with open(path+filename+'.pkl', 'wb') as handle:
        pickle.dump(dict_, handle)


In [25]:
store_bow_files(train,"train_dict")
store_bow_files(test,"test_dict")
store_bow_files(test_1,"test_1_dict")
store_bow_files(test_2,"test_2_dict")
store_bow_files(valid,"valid_dict")

## Validating The stored Files

In [26]:
with open(path + 'train_dict.pkl', 'rb') as handle:
    train = pickle.load(handle)
train

{'tokens': [array([[0, 1, 2, 3, 4]]),
  array([[ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
          21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
          37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]]),
  array([[ 2, 12, 21, 22, 37, 41, 43, 44, 48, 50, 51, 52, 53, 54, 55, 56,
          57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
          73, 74, 75, 76, 77, 78, 79, 80]]),
  array([[12, 62, 71, 74, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92,
          93, 94, 95, 96]]),
  array([[ 22,  23,  41,  49,  96,  97,  98,  99, 100, 101, 102, 103, 104,
          105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
          118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
          131, 132, 133, 134, 135, 136, 137, 138, 139]]),
  array([[  2,  12,  41,  48,  93, 140, 141, 142, 143, 144, 145, 146, 147,
          148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
          161, 162]]

In [27]:
with open(path + 'test_dict.pkl', 'rb') as handle:
    test = pickle.load(handle)
test

{'tokens': [array([[0, 1, 2, 3, 4]]),
  array([[ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
          21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
          37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]]),
  array([[ 2, 12, 21, 22, 37, 41, 43, 44, 48, 50, 51, 52, 53, 54, 55, 56,
          57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
          73, 74, 75, 76, 77, 78, 79, 80]]),
  array([[12, 62, 71, 74, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92,
          93, 94, 95, 96]]),
  array([[ 22,  23,  41,  49,  96,  97,  98,  99, 100, 101, 102, 103, 104,
          105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
          118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
          131, 132, 133, 134, 135, 136, 137, 138, 139]]),
  array([[  2,  12,  41,  48,  93, 140, 141, 142, 143, 144, 145, 146, 147,
          148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
          161, 162]]

In [28]:
with open(path + 'valid_dict.pkl', 'rb') as handle:
    valid = pickle.load(handle)
valid

{'tokens': [array([[   12,    22,    27,    43,    45,    48,    75,   152,   161,
            203,   283,   377,   404,   504,   743,   758,   792,   957,
            967,  1030,  1134,  1184,  1305,  1344,  1983,  2459,  3047,
           4021,  5047,  7961, 10013, 14709, 18769, 18770]]),
  array([[    2,    13,    22,    23,    48,    52,    63,    75,    89,
             93,    96,   102,   109,   114,   152,   182,   188,   195,
            196,   202,   263,   276,   287,   311,   325,   335,   375,
            389,   404,   586,   613,   655,   665,   667,   737,   866,
            890,   983,   988,  1014,  1302,  1327,  1452,  1485,  1494,
           2683,  2834,  3722, 17241]]),
  array([[   2,   26,   30,   43,   48,  129,  152,  154,  178,  211,  278,
           327,  448,  590,  720,  802,  883, 1260, 1682, 1839, 2239, 2639,
          3108, 3223, 3727]]),
  array([[  74,   93,  201,  220,  250,  287,  295,  376,  573,  574,  600,
           932, 2115]]),
  array([[ 16,  89,